# Read Toloka result tsv files into pandas

In [1]:
import requests
import pandas as pd
import json


r_config = requests.get('https://raw.githubusercontent.com/od-crypto/somedata/master/toloka_results/toloka_config.json')


toloka_config_file = 'toloka_config.json'
with open(toloka_config_file, 'w') as f:
    f.write(r_config.text)

with open(toloka_config_file, 'r') as f:
    config = json.load(f)

    
r1 = requests.get(config['task1_file'])
r2 = requests.get(config['task2_file'])


with open('toloka-1.tsv', 'w') as f:
    f.write(r1.text)

with open('toloka-2.tsv', 'w') as f:
    f.write(r2.text)

In [2]:
df_accepted = pd.read_csv('toloka-1.tsv', sep='\t')
df_check = pd.read_csv('toloka-2.tsv', sep='\t')

### Manually: Toloka json answers to be excluded: 

In [3]:
remove_from_accepted = config['remove_from_accepted']
remove_from_task2 = config['remove_from_task2']
# incorrect links in the second Toloka task:
broken_task2 = config['broken_task2']

accepted_results = df_accepted.loc[:,['OUTPUT:result']]
accepted_images = df_accepted.loc[:,'INPUT:image'].to_list()
check_images = df_check.loc[:,'INPUT:image'].to_list()

print("remove manually from accepted (1st task): ", len(remove_from_accepted))
print("accepted_images (1st task): ", len(df_accepted), len(accepted_images), len(set(accepted_images)) )
print("check_images (2nd task): ", len(df_check), len(check_images), len(set(check_images)) )

remove_check2 = remove_from_task2 + broken_task2
print(len(df_check), len(set(remove_check2)) )

print("Total good images: ", len(df_accepted) - len(set(remove_from_accepted)) + len(df_check) - len(set(remove_check2)))

remove manually from accepted (1st task):  53
accepted_images (1st task):  195 195 195
check_images (2nd task):  86 86 86
86 17
Total good images:  212


In [4]:
df_good2 = df_check[~df_check['INPUT:image'].isin(remove_check2)]

good_list2 = df_good2.loc[:,'INPUT:image'].to_list()
print("Items in good_list2: ", len(df_good2), len(set(good_list2)))

set(remove_check2) - set(check_images)

Items in good_list2:  69 69


set()

In [5]:
df_good1 = df_accepted[~df_accepted['INPUT:image'].isin(remove_from_accepted)]

good_list1 = df_good1.loc[:,'INPUT:image'].to_list()
print("Items in good_list1 (1st task): ", len(df_good1), len(set(good_list1)))

print(len(df_accepted)-len(remove_from_accepted), len(set(remove_from_accepted) - set(accepted_images)))

set(remove_from_accepted) - set(accepted_images)

Items in good_list1 (1st task):  143 143
142 0


set()

### Create the masks (+ collages) from Toloka json answers: 

In [6]:
#! pip install demjson

In [7]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [9]:
#from PIL import Image, ImageDraw
import requests
import numpy as np
from urllib.request import urlopen
#from StringIO import StringIO
from io import StringIO
from io import BytesIO
import PIL.Image
import cv2
import os

from create_masks import create_poly, plot_locally, create_collage, create_masks

dconfig_name = '../dataset_config.json'
with open(dconfig_name, 'r') as f:
    dconfig = json.load(f)

dconfig['landcover']

SIMAGES = '../' + dconfig['sentinel']['image_dir']
SCOLLAGES = '../' + dconfig['sentinel']['collage_dir']
SMASKS =  '../' + dconfig['sentinel']['mask_dir']

SIMAGES, SCOLLAGES, SMASKS

df_final = pd.concat([df_good1, df_good2])
print(len(df_final))

df = df_final
create_masks(df, SIMAGES, SCOLLAGES, SMASKS, False, 0, len(df))

212
Create masks for images:  1  to  212
Total masks created:  212
